In [109]:
import json
import requests
from dotenv import load_dotenv
import os
import datetime
load_dotenv()

BASE_URL = 'https://www.courtlistener.com/api/rest/v3/'
HEADERS = {'Authorization': 'Token {}'.format(os.getenv("CL_API_KEY"))}
CAFC_OPINIONS = "data/cafc_opinions.json"
CAFC_HEARINGS = "data/cafc_hearings.json"

## Collect CAFC Opinions

In [115]:
with open(CAFC_OPINIONS, "r") as read_file:
    data = json.load(read_file)
    
uri_list = [opinion["resource_uri"] for opinion in data]

response = requests.get(BASE_URL + 'opinions/?cluster__docket__court__id=cafc', headers=HEADERS)
opinions = response.json()["results"]
next_page = response.json()["next"] # Not Currently Being Used
for i, opinion in enumerate(opinions):
    cluster = requests.get(opinion["cluster"], headers=HEADERS)
#     print (cluster.json()["case_name"])
    opinion_data = {
        "resource_uri": opinion["resource_uri"],
        "date_created": datetime.datetime.strptime(opinion["date_created"], "%Y-%m-%dT%H:%M:%S.%f%z").strftime('%m/%d/%y'),
        "date_modified": opinion["date_modified"],
        "page_count": opinion["page_count"],
        "download_url": opinion["download_url"],
        "local_path": opinion["local_path"],
        "plain_text": opinion["plain_text"],
        "opinions_cited": opinion["opinions_cited"],
        "case_name": cluster.json()["case_name"]
    }
    if opinion_data["resource_uri"] not in uri_list:
        uri_list.append(opinion["resource_uri"])
        data.append(opinion_data)

with open(CAFC_OPINIONS, 'w') as outfile:
    json.dump(data, outfile)

## Collect CAFC Hearings

In [116]:
with open("data/cafc_hearings.json", "r") as read_file:
    data = json.load(read_file)
    
uri_list = [hearing["resource_uri"] for hearing in data]

response = requests.get(BASE_URL + 'audio/?cluster__docket__court__id=cafc', headers=HEADERS)
hearings = response.json()["results"]
for i, hearing in enumerate(hearings):
    hearing_data = {
        "resource_uri": hearing["resource_uri"],
        "date_created": datetime.datetime.strptime(hearing["date_created"], "%Y-%m-%dT%H:%M:%S.%f%z").strftime('%m/%d/%y'),
        "date_modified": hearing["date_modified"],
        "download_url": hearing["download_url"],
        "local_path_mp3": hearing["local_path_mp3"],
        "duration": hearing["duration"],
        "case_name": hearing["case_name"]
    }
    if hearing_data["resource_uri"] not in uri_list:
        uri_list.append(hearing['resource_uri'])
        data.append(hearing_data)
        
with open(CAFC_HEARINGS, 'w') as outfile:
    json.dump(data, outfile)

In [102]:
import pandas as pd


df2 = pd.DataFrame(r2.json()['results'])

In [118]:
# df2.columns.values

In [117]:
# df2.head()